In [7]:
import requests
from bs4 import BeautifulSoup
import time
import random


In [176]:
url = "https://www.mobile01.com/waypointtopiclist.php?sort=topictime&f=189&p=3"


In [168]:
proxylist = [
             {"https": "206.189.36.198:8080"},
             {"http": "37.187.120.123:80"},
             {"https": "178.128.31.153:8080"},
             {"http": "167.114.180.102:8080"},
             {"http": "104.131.214.218:80"},
             {"http": "167.114.196.153:80"}]
header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}



def get_connect(url):
    i = 4
    resp = ""
    try :
        resp = requests.get(url,headers=header,proxies=proxylist[i])
        resp.encoding = "utf-8"
        time.sleep(random.randint(1,4))
        # 設定連線狀況 log
        code = "HTTP response code = " + str(resp.status_code)
        #logger.info(code,extra={"url" : url, "time_use" : resp.elapsed.total_seconds()})
        while resp.status_code != 200 :
            #logger.warning("Connect error",extra={"url" : url, "response": resp.raise_for_status()})
            resp.close()
            i = (i + 1) % 6
            resp = requests.get(url,headers=header,proxies=proxylist[i])
            resp.encoding = "utf-8"



    except Exception as e:
        #logger.error(e,exc_info=True)
        print(e)
    
    return resp.text
    

In [36]:
# 進入 M1 取得 List 標題 超連結 回文數
soup = BeautifulSoup(get_connect(url))
article_list = []
for post in soup.find_all("tr")[1:]:
    
    relpy_int = int(post.find("td", class_="reply").text)
    subject = post.find("a",class_="topic_gen")
    href = "https://www.mobile01.com/" + subject["href"]
    title = subject.text
    article_list.append({
        "title" : title,
        "href" : href,
        "reply_count" : relpy_int
    })

In [129]:
urls = "https://www.mobile01.com/waypointtopicdetail.php?f=189&t=5680477"

In [130]:
# 進入Ｍ1 List中 每篇文章 取得 作者 時間 內文 評論者 評論內容

soup = BeautifulSoup(get_connect(urls))
content = soup.find(id = "forum-content-main")

In [132]:
article_title = content.find("h1", class_="topic").text
first = content.find("article")
author = first.find("div", class_="fn").find("a").text
hot = first.find("div", class_="info").text
date = first.find("div", class_="date").text
first.find("div", class_="single-post-author group").extract()
main_content = first.text

In [160]:
comments = []
for single_post in content.find_all("article")[1:]:
    author = single_post.find("div", class_="fn").find("a").text
    #single_post.find("blockquote").extract()
    comment = single_post.find("div", class_="single-post-content").text
    comment = author + " : " + comment
    comments.append(comment)

In [161]:
comments

['Dr.sam : \n\n\n\r\n樓主，發錯版。這邊是台東版。\n\n',
 'artist7 : \n尷尬...無法修改，只好等01更正了...感謝告知！\n\n\n',
 'sige777 : \n看起來好吃！\n\n',
 'XaosLin : \n\n\r\n幫你按一下回報，不然應該等到天荒地老都還在台東\n心頭滅却\n',
 'artist7 : \n感謝！\n\n',
 'artist7 : \n收到移動文章通知了！\n\n',
 'artist7 : \n被移了！\n\n']

In [175]:
# 組合前面 寫成流程

def parse(url):
    print("進入 :" ,url)
    soup = BeautifulSoup(get_connect(url))
    time.sleep(random.randint(1,5))
    result = []
    content_in_post = []
    global relpy_int, href, title, author , hot , date , main_content
    for post in soup.find_all("tr")[1:]:

        relpy_int = int(post.find("td", class_="reply").text)
        subject = post.find("a",class_="topic_gen")
        href = "https://www.mobile01.com/" + subject["href"]
        title = subject.text
        
        # connect into evey post in list
        print("處理中 : " ,href)
        time.sleep(random.randint(3,7))
        try:
            soup = BeautifulSoup(get_connect(href))
        except Exception as e:
            print(e)
            pass
        if soup :
            content = soup.find(id = "forum-content-main")
            article_title = content.find("h1", class_="topic").text
            first = content.find("article")
            author = first.find("div", class_="fn").find("a").text
            hot = first.find("div", class_="info").text
            date = first.find("div", class_="date").text
            first.find("div", class_="single-post-author group").extract()
            main_content = first.text
            comments = []
            for single_post in content.find_all("article")[1:]:
                author = single_post.find("div", class_="fn").find("a").text
                #single_post.find("blockquote").extract()
                comment = single_post.find("div", class_="single-post-content").text
                comment = author + " : " + comment
                comments.append(comment)
        
    
        # append all
        result.append({
            "title" : title,
            "author" : author,
            "hot" : hot,
            "date" : date,
            "href" : href,
            "reply" : relpy_int,
            "main_content" : main_content,
            "comment" : comments

        })
    print("work done")
    return result
    
    

In [177]:
parse(url)

進入 : https://www.mobile01.com/waypointtopiclist.php?sort=topictime&f=189&p=3
處理中 :  https://www.mobile01.com/waypointtopicdetail.php?f=189&t=5659588
處理中 :  https://www.mobile01.com/waypointtopicdetail.php?f=189&t=5659422
處理中 :  https://www.mobile01.com/waypointtopicdetail.php?f=189&t=5659286
處理中 :  https://www.mobile01.com/waypointtopicdetail.php?f=189&t=5659178
處理中 :  https://www.mobile01.com/waypointtopicdetail.php?f=189&t=5658456
處理中 :  https://www.mobile01.com/waypointtopicdetail.php?f=189&t=5658438
處理中 :  https://www.mobile01.com/waypointtopicdetail.php?f=189&t=5658321
處理中 :  https://www.mobile01.com/waypointtopicdetail.php?f=189&t=5657389
處理中 :  https://www.mobile01.com/waypointtopicdetail.php?f=189&t=5656931
處理中 :  https://www.mobile01.com/waypointtopicdetail.php?f=189&t=5656626
處理中 :  https://www.mobile01.com/waypointtopicdetail.php?f=189&t=5656446
處理中 :  https://www.mobile01.com/waypointtopicdetail.php?f=189&t=5656079
處理中 :  https://www.mobile01.com/waypointtopicdetail.php?f=1

[{'title': '【誠品南西店】 金品茶語  X 珍珠奶茶鬆餅 不甜不膩 超推薦',
  'author': '花惹發WTF12345',
  'hot': '文章人氣: 8,173 ',
  'date': '2018-12-11 22:09\xa0\xa0#1',
  'href': 'https://www.mobile01.com/waypointtopicdetail.php?f=189&t=5659588',
  'reply': 0,
  'main_content': '\n\n\n\n#台北 #美食 #金品茶語\n\r\n🔸 台灣特色小吃一網打盡：小籠包、牛肉麵、珍珠奶茶 \r\n🔸 烏龍茶品 品茶饗宴 \r\n🔸 珍珠奶茶鬆餅 不甜不膩 超推薦 \n\r\n不論是多汁的小籠包，還是茶香四溢的飲品\r\n很推薦大家試試啊😂\n\r\n👉吃貨圖文分享 \n\r\n- - - - - - - - - - - - - - - - - - - -\r\n📍 金品茶語：誠品南西店\r\n🏠台北市中山區南京西路14號4樓\r\n☎️(02)2521 9669\r\n🕘 營業時間：周日~周四11:00 ~ 22:00\r\n             周五~周六11:00 ~ 22:30\n\n\n\n\n\n\n\n\n\n',
  'comment': []},
 {'title': '【台北市 內湖區/近內湖Costco】燒肉的名門 赤虎-大口品嚐日式燒肉超爽快',
  'author': 'allyyang701',
  'hot': '文章人氣: 10,824 ',
  'date': '2018-12-11 20:06\xa0\xa0#1',
  'href': 'https://www.mobile01.com/waypointtopicdetail.php?f=189&t=5659422',
  'reply': 0,
  'main_content': '\n\n\n\n文章出處Ally 愛玩樂http://ally701.pixnet.net/blog/post/466537493-%E3%80%90%E5%8F%B0%E5%8C%97%E5%B8%82-%E5%85%A7%E6%B9%96%E5%8D%80-%E8%BF%91%E5%8

NameError: name 'result' is not defined